In [46]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms

In [47]:
transform = transforms.Compose([transforms.ToTensor()])
mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

batch_size = 32
#split train, val, test using train_test_split
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(mnist_data, test_size= 0.3, random_state=42)
val_data, test_data = train_test_split(val_data,test_size=0.5, random_state=42)

train_loader_pytorch = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader_pytorch = torch.utils.data.DataLoader(val_data,batch_size=batch_size, shuffle=True)
test_loader_pytorch = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)



In [48]:
# Baseline model with Convolutional model
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,32,3,1)
        self.pool1 = torch.nn.MaxPool2d(2,2)
        self.conv2 = torch.nn.Conv2d(32,64,3,1)
        self.pool2 = torch.nn.MaxPool2d(2,2)
        self.fc1 = torch.nn.Linear(1600,128)
        self.fc2 = torch.nn.Linear(128,10)
        self.relu = torch.nn.ReLU()
    def forward(self,X):
        X = self.conv1(X)
        X = self.relu(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = self.relu(X)
        X = self.pool2(X)
        X = torch.flatten(X,1)
        X = self.fc1(X)
        X = self.relu(X)
        X = self.fc2(X)
        return X


In [ ]:
# Training loop
import tqdm
criterion = torch.nn.CrossEntropyLoss()
from optimizers import StochasticArmijoSGD

num_epochs = 20  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for epoch in range(num_epochs):
    for optimizer_name in ["Armijo", "Adam"]:
        #reset model weights
        model = ConvNet()

        if optimizer_name == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        else:
            optimizer = StochasticArmijoSGD(model.parameters(), initial_step=1.0, c=0.001, tau=0.5, max_backtracks=10)

        for images, labels in tqdm.tqdm(train_loader_pytorch):
            images, labels = images.to(device), labels.to(device)
            if optimizer_name == "Adam":
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            else:  # armijo
                def closure(backward=True):
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    if backward:
                        loss.backward()
                    return loss
                optimizer.step(closure)
        # validation
        model.eval()
        val_loss = 0
        correct =0 
        total= 0
        with torch.no_grad():
            for images, labels in val_loader_pytorch:
                outputs = model(images)
                loss = criterion(outputs,labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data,1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            val_loss = val_loss / len(val_loader_pytorch)
            val_accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Optimizer: {optimizer_name}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')



  0%|          | 0/1313 [00:00<?, ?it/s]

100%|██████████| 1313/1313 [00:44<00:00, 29.77it/s]


Epoch [1/20], Optimizer: Armijo, Val Loss: 2.3052, Val Accuracy: 0.1131


100%|██████████| 1313/1313 [00:33<00:00, 39.52it/s]


Epoch [1/20], Optimizer: Adam, Val Loss: 0.0848, Val Accuracy: 0.9749


...

100%|██████████| 1313/1313 [00:31<00:00, 41.84it/s]


Epoch [3/20], Optimizer: Armijo, Val Loss: 2.3069, Val Accuracy: 0.1007


100%|██████████| 1313/1313 [00:25<00:00, 51.22it/s]


Epoch [3/20], Optimizer: Adam, Val Loss: 0.0817, Val Accuracy: 0.9731


 78%|███████▊  | 1026/1313 [00:26<00:07, 36.97it/s]